#### Prerequisites

In [1]:
%%capture 

!pip install sagemaker==2.100.0
!pip install jedi==0.17  # This is a requirement for pygmentize to work

#### Imports 

In [2]:
from sagemaker.huggingface import HuggingFaceProcessor
from sagemaker.processing import ProcessingOutput
from sagemaker.processing import ProcessingInput
from sagemaker import get_execution_role
from sagemaker import Session
import sagemaker
import logging

##### Setup logging

In [3]:
logger = logging.getLogger('sagemaker')
logger.setLevel(logging.DEBUG)
logger.addHandler(logging.StreamHandler())

##### Log versions of dependencies

In [4]:
logger.info(f'[Using SageMaker version: {sagemaker.__version__}]')

[Using SageMaker version: 2.100.0]


#### Essentials 

In [5]:
session = Session()
S3_BUCKET = session.default_bucket()
logger.info(f'S3 bucket = {S3_BUCKET}')
ROLE = get_execution_role()

S3_INPUT = 'data/covid_articles.txt'
S3_VOCAB = 'data/bert/vocab/vocab.txt'
S3_OUTPUT = 'data/bert/processed'
INSTANCE_TYPE = 'ml.g4dn.xlarge'
INSTANCE_COUNT = 1
PYTHON_VERSION = '1.6.0'
TRANSFORMERS_VERSION = '4.4.2'
BASE_JOB_NAME = 'hf-sagemaker-processor'
SOURCE_DIR = './src'
CODE = 'preprocess_data_oob.py'

S3 bucket = sagemaker-us-east-1-119174016168


#### Download vocabulary for original BERT base uncased to local `vocab` directory

In [6]:
%%capture

!wget -q https://s3.amazonaws.com/models.huggingface.co/bert/bert-base-uncased-vocab.txt -O ./vocab/vocab.txt

##### Copy vocabulary file from local `vocab` directory to S3 

In [7]:
!aws s3 cp ./vocab/vocab.txt s3://{S3_BUCKET}/data/bert/vocab/vocab.txt


The user-provided path ./vocab/vocab.txt does not exist.


#### View processing script

In [8]:
!pygmentize -v ./src/preprocess_data_oob.py

from transformers import BertTokenizerFast
from datasets import load_dataset
from datasets import DatasetDict
from pathlib import Path
import transformers 
import datasets
import logging
import sys
import os

# Setup logging
logger = logging.getLogger(__name__)
logging.basicConfig(level=logging.getLevelName('INFO'), 
                    handlers=[logging.StreamHandler(sys.stdout)], 
                    format='%(asctime)s - %(name)s - %(levelname)s - %(message)s')

# Log versions of dependencies
logger.info(f'[Using Transformers: {transformers.__version__}]')
logger.info(f'[Using Datasets: {datasets.__version__}]')

# Essentials
# LOCAL_INPUT_PATH is mapped to S3 input location for covid articles 
LOCAL_INPUT_PATH = '/opt/ml/processing/input' 
# LOCAL_OUTPUT_PATH is mapped to S3 output location where we want to save the processed input data (COVID articles)
LOCAL_OUTPUT_PATH = '/opt/ml/processing/output'
MAX_LENGTH = 512
CHUNK_SIZE = 128
N_GPUS = 1

#logger.info(f'Re-creating BERT toke

#### Create HuggingFace Processor

In [9]:
processor = HuggingFaceProcessor(role=ROLE, 
                                 instance_type=INSTANCE_TYPE, 
                                 instance_count=INSTANCE_COUNT, 
                                 transformers_version=TRANSFORMERS_VERSION, 
                                 pytorch_version=PYTHON_VERSION, 
                                 base_job_name=BASE_JOB_NAME)

#### Run SageMaker Processing Job

In [ ]:
%%time

processor.run(code=CODE, 
              source_dir=SOURCE_DIR,
              inputs=[ProcessingInput(input_name='articles', 
                                      source=f's3://{S3_BUCKET}/{S3_INPUT}', 
                                      destination='/opt/ml/processing/input/data'),
                      ProcessingInput(input_name='bert-vocab',
                                      source=f's3://{S3_BUCKET}/{S3_VOCAB}',
                                      destination='/opt/ml/processing/input/vocab')
                     ],
              outputs=[ProcessingOutput(output_name='processed-data-oob', 
                                        source='/opt/ml/processing/output', 
                                        destination=f's3://{S3_BUCKET}/{S3_OUTPUT}')])

Uploaded ./src to s3://sagemaker-us-east-1-119174016168/hf-sagemaker-processor-2022-09-01-05-13-04-518/source/sourcedir.tar.gz
runproc.sh uploaded to s3://sagemaker-us-east-1-119174016168/hf-sagemaker-processor-2022-09-01-05-13-04-518/source/runproc.sh
Creating processing-job with name hf-sagemaker-processor-2022-09-01-05-13-04-518
process request: {
    "ProcessingJobName": "hf-sagemaker-processor-2022-09-01-05-13-04-518",
    "ProcessingResources": {
        "ClusterConfig": {
            "InstanceType": "ml.g4dn.xlarge",
            "InstanceCount": 1,
            "VolumeSizeInGB": 30
        }
    },
    "AppSpecification": {
        "ImageUri": "763104351884.dkr.ecr.us-east-1.amazonaws.com/huggingface-pytorch-training:1.6.0-transformers4.4.2-gpu-py36-cu110-ubuntu18.04",
        "ContainerEntrypoint": [
            "/bin/bash",
            "/opt/ml/processing/input/entrypoint/runproc.sh"
        ]
    },
    "RoleArn": "arn:aws:iam::119174016168:role/service-role/AmazonSageMaker-Ex


Job Name:  hf-sagemaker-processor-2022-09-01-05-13-04-518
Inputs:  [{'InputName': 'articles', 'AppManaged': False, 'S3Input': {'S3Uri': 's3://sagemaker-us-east-1-119174016168/data/covid_articles.txt', 'LocalPath': '/opt/ml/processing/input/data', 'S3DataType': 'S3Prefix', 'S3InputMode': 'File', 'S3DataDistributionType': 'FullyReplicated', 'S3CompressionType': 'None'}}, {'InputName': 'bert-vocab', 'AppManaged': False, 'S3Input': {'S3Uri': 's3://sagemaker-us-east-1-119174016168/data/bert/vocab/vocab.txt', 'LocalPath': '/opt/ml/processing/input/vocab', 'S3DataType': 'S3Prefix', 'S3InputMode': 'File', 'S3DataDistributionType': 'FullyReplicated', 'S3CompressionType': 'None'}}, {'InputName': 'code', 'AppManaged': False, 'S3Input': {'S3Uri': 's3://sagemaker-us-east-1-119174016168/hf-sagemaker-processor-2022-09-01-05-13-04-518/source/sourcedir.tar.gz', 'LocalPath': '/opt/ml/processing/input/code/', 'S3DataType': 'S3Prefix', 'S3InputMode': 'File', 'S3DataDistributionType': 'FullyReplicated', '